In [1]:
using Optim
using DataFrames;using CSV
using Pkg
using LinearAlgebra;
using JuliaZH;JuliaZH.set_mirror("PKU")#JuliaZH.generate_startup("BFSU")
ccall((:openblas_get_num_threads64_, Base.libblas_name),Cint,())
LinearAlgebra.BLAS.set_num_threads(8)
using Base.Threads;nthreads()
using Meteor.ExactDiag;
using JLD2;using FileIO
using DifferentialEquations

In [2]:
function dm2vec(dm)
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,3,:)
end
function thermal_state(dims,nbar)
    c::Array{Complex{Float64}}=zeros(dims,dims)
    if nbar==0
        c[1,1]=1
    else
        for i in 1:dims
           c[i,i]= exp(-i*log(1/nbar+1))
        end
    end
    c=c/tr(c)
    return c        
end
    
function coherence_state(alpha)
    m=exp(alpha*a₊(dim)-alpha'*a₋(dim))*n0(dim)
    return  m*m'
end
    
function fock_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    if n<dims
        c[n+1,n+1]=1
    end
    return c
end
    
function equ_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    for i in 1:n
        c[i,i]=1/n    
    end
    return c
end
    
dim=4
η=0.084 
    
function a₋(n)      #n为维数
    s::Array{Complex{Float64}}=zeros(n,n)
    for i=1:n-1
        s[i,i+1]=sqrt(i)
    end
    return s
end

a₊(n)=adjoint(a₋(n));
a=a₊(dim)+a₋(dim)    
a2=kron([0 0;1 0],exp(im*η*a))
a3=kron([0 1;0 0],exp(-im*η*a))
# a2=kron([0 0;1 0],im*η*a₊(dim))
# a3=kron([0 1;0 0],-im*η*a₋(dim))   
#经过尝试发现,符合脉冲并不精准,都保留下来就很不精确
    
σ₊=[0 0;1 0]
σ₋=[0 1;0 0]
σ_x=[0 1;1 0]
σ_y=[0 -1im;1im 0]
σ_z=[1 0;0 -1]

function dm2vec(dm)
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,Int(sqrt(size(vec)[1])),:)
end
    
function temperature(nbar)
    return (6.626*1e-34*1e6*1)/(1.38*1e-23*log(1/nbar+1))
end
    
n=kron(I(2),a₊(dim)*a₋(dim))
en=kron([0 0;0 1],I(dim))

ν=1.
    
function H(Ω,Δ)
    unitary::Matrix{ComplexF64}=kron(I(dim*2),-im*(ν*n-Δ*en+Ω/2*a2+Ω'/2*a3))+kron(transpose(im*(ν*n-Δ*en+Ω/2*a2+Ω'/2*a3)),I(dim*2))
    return unitary
end
    
function Hv(Ω,Δ)
    unitary::Matrix{ComplexF64}=ν*n-Δ*en+Ω/2*a2+Ω'/2*a3
    return unitary
end


Hv (generic function with 1 method)

In [3]:
function R(t,phi,omega,delta)
    return exp(-im*t*Hv(omega*exp(im*phi),delta))
end

R (generic function with 1 method)

In [4]:
function swap_up(dims,n)#下态到上态就行
    
    s=zeros(dims,dims)+I(dims)
    s[n+1,n+1]=0.;
   
    
    return s
end
swap=swap_up(dim,0)

4×4 Matrix{Float64}:
 0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0

In [5]:
# function swap_prod(dims,n)#蓝边带交换
#     s=zeros(dim*2,dim*2)+I(dim*2)
#     s[n+1,n+1]=0.;
#     s[n+2+dims,n+2+dims]=0.;
#     s[n+2+dims,n+1]=1.;
#     s[n+1,n+2+dims]=1.;
    
    
#     return s
# end
# swap=swap_prod(dim,2)

In [6]:
function feval(x)
    swapfit=R(x[4],x[6],1,x[1])*R(x[3],x[5],1,x[1])*R(x[2],0,1,x[1])
    opres=norm(norm.(swapfit[1:dim,1:dim])-norm.(swap))
    println(round.(x,digits=5),"    ",round(opres,digits=4))    
    return opres
end

feval (generic function with 1 method)

In [18]:
xini=[0.93062, 148.1263, 149.59965, 140.62509, 1.09029, 6.28319]
# xini=[0.93126, 148.1721, 149.50025, 140.63154, 1.11823, 6.37707]#feom0
lower=[0.5,0,0,0,0,0];upper=[2.5,4*pi/η,4*pi/η,4*pi/η,2*pi,2*pi];
# res=optimize(feval,xini,ParticleSwarm(; lower = lower,upper = upper,n_particles = 10),Optim.Options(iterations = 10000))
res= optimize(feval,xini,LBFGS()) 

[0.93063, 148.1263, 149.59965, 140.62509, 1.09029, 6.28319]    0.2691
[0.93061, 148.1263, 149.59965, 140.62509, 1.09029, 6.28319]    0.2691
[0.93062, 148.1272, 149.59965, 140.62509, 1.09029, 6.28319]    0.2691
[0.93062, 148.1254, 149.59965, 140.62509, 1.09029, 6.28319]    0.2691
[0.93062, 148.1263, 149.60056, 140.62509, 1.09029, 6.28319]    0.2691
[0.93062, 148.1263, 149.59874, 140.62509, 1.09029, 6.28319]    0.2691
[0.93062, 148.1263, 149.59965, 140.62594, 1.09029, 6.28319]    0.2691
[0.93062, 148.1263, 149.59965, 140.62424, 1.09029, 6.28319]    0.2691
[0.93062, 148.1263, 149.59965, 140.62509, 1.0903, 6.28319]    0.2691
[0.93062, 148.1263, 149.59965, 140.62509, 1.09028, 6.28319]    0.2691
[0.93062, 148.1263, 149.59965, 140.62509, 1.09029, 6.28323]    0.2691
[0.93062, 148.1263, 149.59965, 140.62509, 1.09029, 6.28315]    0.2691
[0.93062, 148.1263, 149.59965, 140.62509, 1.09029, 6.28319]    0.2691
[1.08967, 148.12693, 149.58954, 140.62571, 1.09031, 6.29456]    0.9547
[1.08966, 148.12693,

[0.93062, 148.1254, 149.59964, 140.62509, 1.09029, 6.2832]    0.2691
[0.93062, 148.1263, 149.60055, 140.62509, 1.09029, 6.2832]    0.2691
[0.93062, 148.1263, 149.59874, 140.62509, 1.09029, 6.2832]    0.2691
[0.93062, 148.1263, 149.59964, 140.62594, 1.09029, 6.2832]    0.2691
[0.93062, 148.1263, 149.59964, 140.62424, 1.09029, 6.2832]    0.2691
[0.93062, 148.1263, 149.59964, 140.62509, 1.0903, 6.2832]    0.2691
[0.93062, 148.1263, 149.59964, 140.62509, 1.09028, 6.2832]    0.2691
[0.93062, 148.1263, 149.59964, 140.62509, 1.09029, 6.28324]    0.2691
[0.93062, 148.1263, 149.59964, 140.62509, 1.09029, 6.28316]    0.2691
[0.93062, 148.1263, 149.59964, 140.62509, 1.09029, 6.2832]    0.2691
[0.93063, 148.1263, 149.59962, 140.62509, 1.09029, 6.28322]    0.2691
[0.93062, 148.1263, 149.59962, 140.62509, 1.09029, 6.28322]    0.2691
[0.93062, 148.1272, 149.59962, 140.62509, 1.09029, 6.28322]    0.2691
[0.93062, 148.1254, 149.59962, 140.62509, 1.09029, 6.28322]    0.2691
[0.93062, 148.1263, 149.60053

[0.93138, 148.12936, 149.50589, 140.62533, 1.10845, 6.37059]    0.2682
[0.93138, 148.12936, 149.50589, 140.62618, 1.10846, 6.37059]    0.2682
[0.93138, 148.12936, 149.50589, 140.62618, 1.10845, 6.37059]    0.2682
[0.93138, 148.12936, 149.50589, 140.62618, 1.10845, 6.37063]    0.2682
[0.93138, 148.12936, 149.50589, 140.62618, 1.10845, 6.37055]    0.2682
[0.93138, 148.12936, 149.50589, 140.62618, 1.10845, 6.37059]    0.2682
[0.93137, 148.13001, 149.50835, 140.62641, 1.11235, 6.36518]    0.2682
[0.93136, 148.13001, 149.50835, 140.62641, 1.11235, 6.36518]    0.2682
[0.93136, 148.13091, 149.50835, 140.62641, 1.11235, 6.36518]    0.2682
[0.93136, 148.12912, 149.50835, 140.62641, 1.11235, 6.36518]    0.2682
[0.93136, 148.13001, 149.50925, 140.62641, 1.11235, 6.36518]    0.2682
[0.93136, 148.13001, 149.50744, 140.62641, 1.11235, 6.36518]    0.2682
[0.93136, 148.13001, 149.50835, 140.62726, 1.11235, 6.36518]    0.2682
[0.93136, 148.13001, 149.50835, 140.62556, 1.11235, 6.36518]    0.2682
[0.931

[0.93136, 148.13027, 149.50847, 140.6265, 1.1138, 6.36404]    0.2682
[0.93136, 148.13027, 149.50847, 140.6265, 1.1138, 6.36397]    0.2682
[0.93136, 148.13027, 149.50847, 140.6265, 1.1138, 6.36401]    0.2682
[0.93137, 148.13029, 149.50846, 140.62649, 1.1138, 6.36401]    0.2682
[0.93136, 148.13029, 149.50846, 140.62649, 1.1138, 6.36401]    0.2682
[0.93136, 148.13119, 149.50846, 140.62649, 1.1138, 6.36401]    0.2682
[0.93136, 148.12939, 149.50846, 140.62649, 1.1138, 6.36401]    0.2682
[0.93136, 148.13029, 149.50936, 140.62649, 1.1138, 6.36401]    0.2682
[0.93136, 148.13029, 149.50755, 140.62649, 1.1138, 6.36401]    0.2682
[0.93136, 148.13029, 149.50846, 140.62735, 1.1138, 6.36401]    0.2682
[0.93136, 148.13029, 149.50846, 140.62564, 1.1138, 6.36401]    0.2682
[0.93136, 148.13029, 149.50846, 140.62649, 1.11381, 6.36401]    0.2682
[0.93136, 148.13029, 149.50846, 140.62649, 1.11379, 6.36401]    0.2682
[0.93136, 148.13029, 149.50846, 140.62649, 1.1138, 6.36405]    0.2682
[0.93136, 148.13029, 

[0.93135, 148.15931, 149.49494, 140.62149, 1.11351, 6.36703]    0.2681
[0.93136, 148.15898, 149.4954, 140.62154, 1.11343, 6.36672]    0.2681
[0.93135, 148.15898, 149.4954, 140.62154, 1.11343, 6.36672]    0.2681
[0.93135, 148.15988, 149.4954, 140.62154, 1.11343, 6.36672]    0.2681
[0.93135, 148.15808, 149.4954, 140.62154, 1.11343, 6.36672]    0.2681
[0.93135, 148.15898, 149.49631, 140.62154, 1.11343, 6.36672]    0.2681
[0.93135, 148.15898, 149.4945, 140.62154, 1.11343, 6.36672]    0.2681
[0.93135, 148.15898, 149.4954, 140.62239, 1.11343, 6.36672]    0.2681
[0.93135, 148.15898, 149.4954, 140.62069, 1.11343, 6.36672]    0.2681
[0.93135, 148.15898, 149.4954, 140.62154, 1.11344, 6.36672]    0.2681
[0.93135, 148.15898, 149.4954, 140.62154, 1.11343, 6.36672]    0.2681
[0.93135, 148.15898, 149.4954, 140.62154, 1.11343, 6.36676]    0.2681
[0.93135, 148.15898, 149.4954, 140.62154, 1.11343, 6.36668]    0.2681
[0.93135, 148.15898, 149.4954, 140.62154, 1.11343, 6.36672]    0.2681
[0.93136, 148.1593

[0.93129, 148.16676, 149.49566, 140.63109, 1.11442, 6.3708]    0.268
[0.93129, 148.16765, 149.49657, 140.63109, 1.11442, 6.3708]    0.268
[0.93129, 148.16765, 149.49476, 140.63109, 1.11442, 6.3708]    0.268
[0.93129, 148.16765, 149.49566, 140.63194, 1.11442, 6.3708]    0.268
[0.93129, 148.16765, 149.49566, 140.63024, 1.11442, 6.3708]    0.268
[0.93129, 148.16765, 149.49566, 140.63109, 1.11442, 6.3708]    0.268
[0.93129, 148.16765, 149.49566, 140.63109, 1.11441, 6.3708]    0.268
[0.93129, 148.16765, 149.49566, 140.63109, 1.11442, 6.37084]    0.268
[0.93129, 148.16765, 149.49566, 140.63109, 1.11442, 6.37076]    0.268
[0.93129, 148.16765, 149.49566, 140.63109, 1.11442, 6.3708]    0.268
[0.93108, 148.20027, 149.49348, 140.66816, 1.12118, 6.38823]    0.2682
[0.93107, 148.20027, 149.49348, 140.66816, 1.12118, 6.38823]    0.2682
[0.93107, 148.20117, 149.49348, 140.66816, 1.12118, 6.38823]    0.2682
[0.93107, 148.19937, 149.49348, 140.66816, 1.12118, 6.38823]    0.2682
[0.93107, 148.20027, 149

[0.93128, 148.1718, 149.49442, 140.6337, 1.11576, 6.3745]    0.268
[0.93128, 148.1718, 149.49442, 140.63455, 1.11577, 6.3745]    0.268
[0.93128, 148.1718, 149.49442, 140.63455, 1.11576, 6.3745]    0.268
[0.93128, 148.1718, 149.49442, 140.63455, 1.11576, 6.37454]    0.268
[0.93128, 148.1718, 149.49442, 140.63455, 1.11576, 6.37446]    0.268
[0.93128, 148.1718, 149.49442, 140.63455, 1.11576, 6.3745]    0.268
[0.93128, 148.17088, 149.49523, 140.63461, 1.11521, 6.37293]    0.268
[0.93127, 148.17088, 149.49523, 140.63461, 1.11521, 6.37293]    0.268
[0.93127, 148.17178, 149.49523, 140.63461, 1.11521, 6.37293]    0.268
[0.93127, 148.16999, 149.49523, 140.63461, 1.11521, 6.37293]    0.268
[0.93127, 148.17088, 149.49614, 140.63461, 1.11521, 6.37293]    0.268
[0.93127, 148.17088, 149.49433, 140.63461, 1.11521, 6.37293]    0.268
[0.93127, 148.17088, 149.49523, 140.63546, 1.11521, 6.37293]    0.268
[0.93127, 148.17088, 149.49523, 140.63376, 1.11521, 6.37293]    0.268
[0.93127, 148.17088, 149.49523,

[0.93126, 148.1721, 149.50025, 140.63154, 1.11823, 6.37707]    0.268
[0.93127, 148.1721, 149.50025, 140.63155, 1.11823, 6.37707]    0.268
[0.93126, 148.1721, 149.50025, 140.63155, 1.11823, 6.37707]    0.268
[0.93126, 148.173, 149.50025, 140.63155, 1.11823, 6.37707]    0.268
[0.93126, 148.17121, 149.50025, 140.63155, 1.11823, 6.37707]    0.268
[0.93126, 148.1721, 149.50115, 140.63155, 1.11823, 6.37707]    0.268
[0.93126, 148.1721, 149.49934, 140.63155, 1.11823, 6.37707]    0.268
[0.93126, 148.1721, 149.50025, 140.6324, 1.11823, 6.37707]    0.268
[0.93126, 148.1721, 149.50025, 140.6307, 1.11823, 6.37707]    0.268
[0.93126, 148.1721, 149.50025, 140.63155, 1.11824, 6.37707]    0.268
[0.93126, 148.1721, 149.50025, 140.63155, 1.11822, 6.37707]    0.268
[0.93126, 148.1721, 149.50025, 140.63155, 1.11823, 6.37711]    0.268
[0.93126, 148.1721, 149.50025, 140.63155, 1.11823, 6.37703]    0.268
[0.93126, 148.1721, 149.50025, 140.63155, 1.11823, 6.37707]    0.268
[0.93127, 148.1721, 149.50025, 140.6

[0.93126, 148.1721, 149.49934, 140.63154, 1.11823, 6.37707]    0.268
[0.93126, 148.1721, 149.50025, 140.6324, 1.11823, 6.37707]    0.268
[0.93126, 148.1721, 149.50025, 140.63069, 1.11823, 6.37707]    0.268
[0.93126, 148.1721, 149.50025, 140.63154, 1.11824, 6.37707]    0.268
[0.93126, 148.1721, 149.50025, 140.63154, 1.11822, 6.37707]    0.268
[0.93126, 148.1721, 149.50025, 140.63154, 1.11823, 6.37711]    0.268
[0.93126, 148.1721, 149.50025, 140.63154, 1.11823, 6.37703]    0.268
[0.93126, 148.1721, 149.50025, 140.63154, 1.11823, 6.37707]    0.268
[0.93127, 148.1721, 149.50025, 140.63154, 1.11823, 6.37707]    0.268
[0.93126, 148.1721, 149.50025, 140.63154, 1.11823, 6.37707]    0.268
[0.93126, 148.173, 149.50025, 140.63154, 1.11823, 6.37707]    0.268
[0.93126, 148.1712, 149.50025, 140.63154, 1.11823, 6.37707]    0.268
[0.93126, 148.1721, 149.50116, 140.63154, 1.11823, 6.37707]    0.268
[0.93126, 148.1721, 149.49934, 140.63154, 1.11823, 6.37707]    0.268
[0.93126, 148.1721, 149.50025, 140.6

 * Status: success (objective increased between iterations)

 * Candidate solution
    Final objective value:     2.680263e-01

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 5.50e-10 ≰ 0.0e+00
    |x - x'|/|x'|          = 3.68e-12 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.55e-15 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 5.80e-15 ≰ 0.0e+00
    |g(x)|                 = 9.02e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   1  (vs limit Inf)
    Iterations:    22
    f(x) calls:    92
    ∇f(x) calls:   92


In [144]:
# xini

In [19]:
res.minimizer

6-element Vector{Float64}:
   0.9312619523199896
 148.17210183633
 149.50024971267783
 140.63154488456556
   1.1182287922625693
   6.377069693188815

In [20]:
res.minimum

0.2680263365533389

In [21]:
function feval2(x)
    swapfit=R(x[4],x[6],1,x[1])*R(x[3],x[5],1,x[1])*R(x[2],0,1,x[1])
    
    return swapfit
end

feval2 (generic function with 1 method)

In [22]:
round.(DataFrame(abs.(feval2(xini)),:auto),digits=4)

Row,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0945,0.0667,0.0066,0.0011,0.9894,0.0874,0.0048,0.0013
2,0.0391,0.8614,0.0626,0.017,0.0424,0.4145,0.2797,0.0198
3,0.0039,0.0805,0.9024,0.0545,0.0041,0.0689,0.39,0.1388
4,0.0009,0.0036,0.0393,0.8837,0.0003,0.0069,0.0732,0.4606
5,0.9933,0.0236,0.0059,0.0003,0.0976,0.0568,0.0058,0.0007
6,0.0502,0.428,0.016,0.0132,0.0987,0.8918,0.0931,0.0101
7,0.0197,0.2512,0.3964,0.0869,0.0011,0.131,0.8686,0.0134
8,0.0011,0.015,0.1506,0.4562,0.0002,0.005,0.0322,0.8763


In [23]:
# round.(DataFrame(feval2(xini)/(feval2(xini)[1]/abs.(feval2(xini))[1]),:auto),digits=4)

In [24]:
round.(DataFrame(abs.(feval2(res.minimizer)),:auto),digits=4)

Row,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0975,0.0732,0.0071,0.0011,0.9884,0.0902,0.0043,0.0014
2,0.0386,0.8622,0.0583,0.017,0.0473,0.414,0.2783,0.0189
3,0.0042,0.0796,0.9107,0.0477,0.0046,0.0732,0.3708,0.1386
4,0.0009,0.0031,0.049,0.8827,0.0003,0.0072,0.0731,0.4615
5,0.993,0.0255,0.006,0.0004,0.1006,0.0555,0.0059,0.0008
6,0.0501,0.4283,0.021,0.0135,0.1034,0.8907,0.096,0.0105
7,0.0202,0.2461,0.3785,0.088,0.002,0.1364,0.8771,0.0131
8,0.0011,0.0144,0.1452,0.4586,0.0002,0.0051,0.0346,0.8759
